In this notebook is how you can easily start analysing ETH mainnet data by using only open source resources.

Let's start by installing some usuful libraries for the transactions' analysis.
Here the link to the documentation https://web3py.readthedocs.io/en/stable/

In [2]:
!pip install web3==5.25.0
!pip install hexbytes==0.2.2

In [3]:
# !pip install  eth-tester

In order to run the notebook yourself, you may need to create your own account on a node provider website. Here we show you how we do it on Infura: https://github.com/shilpachain/grace_hopper_2022/blob/main/web3_api_demo_for_analytics.ipynb
        

In [4]:
from web3 import Web3
from web3.auto import w3
from hexbytes import HexBytes
import matplotlib.pyplot as plt
import seaborn as sns

Put your url here.

In [5]:
my_url = "https://mainnet.infura.io/v3/df05abee279947a4aaba6515e5bd982a"

In [6]:
# HTTPProvider:
w3 = Web3(Web3.HTTPProvider(my_url))
res = w3.isConnected()
print(res)

Now that we are connected to a node we can access some of the blockchain data. Let's test if the connection is working properly by getting all the transactions in the latest block.

In [7]:
w3.eth.get_block('latest')

For the subsequent analysis let's focus on one single day July 7th 2022.

- First block of July 7th 2022: https://etherscan.io/block/15092101
- Last block of July 7th 2022: https://etherscan.io/block/15098519

- Total blocks: 6418
- Total transactions: 1194064

In [8]:
from datetime import datetime as dt
print(f"{dt.now()}: start")
all_txs = []
for block in range(15092101,15098519):
    block_rpc = w3.eth.get_block(block)
    timestamp = block_rpc['timestamp']
    txs = block_rpc['transactions']
    for tx in txs:
        row = [block, timestamp, w3.toHex(tx)]
        all_txs.append(row)


print(f"Total transaction in range found are {len(all_txs)}")
print(f"{dt.now()}: end")

Let's save all transactions in a pandas dataframe and in a csv file

In [9]:
#all_txs
import pandas as pd
df = pd.DataFrame(data=all_txs, columns=['block_number', 'block_tiemstamp','transaction_hash'])
df.to_csv("all_tx_hashes.csv", index=False)

In [10]:
df

#### Analysis of transactions

Not all transaction's receipts contain a topic. This is probably because some of them are just transfer of ETH between one address and another.

In the following we analyze the first 1000 transactions and get the percentage of pure ETh transfer and percentage of interactions with a smart contract or token transfer.

In [11]:
#get transaction receipt for one transaction
ETH_transfer = []
func_to_add = {}
tx_table = pd.DataFrame(columns=['tx_hash','contract_add','func_call','n_topic','topic_0'], 
                                        index=range(len(df)))
for ind, rows in df.iterrows():
    tx_hash = rows['transaction_hash']
    receipt = w3.eth.get_transaction_receipt(tx_hash)
    tx_info = w3.eth.get_transaction(tx_hash)
    func_call = tx_info['input'][0:10]
    if len(receipt['logs']):
        log = receipt['logs'][0]
        smart_contract = log['address']
        topics = log['topics']
        num_topics = len(topics)
        print(f"Number of topics = {num_topics}")
        topic_0 = w3.toHex(topics[0])
        tx_table.loc[ind] = pd.Series({'tx_hash':tx_hash,'contract_add': smart_contract,'func_call':func_call,
                                            'n_topic': num_topics,'topic_0' : topic_0,})
    else:
        print('This is probably just a ETH transfer')
        ETH_transfer.append(tx_hash)
        tx_table.loc[ind] = pd.Series({'tx_hash':tx_hash,'contract_add': 'nan','func_call':func_call,
                                            'n_topic': 0,'topic_0' : 'nan',})
    
    if ind>1000:
        break

In [12]:
tx_table

Let's give an overview of the amount of topics in each transaction.

Around 350 txs have no topics, meaning they are probably only ETH transfers. Most of the txs have 3 topics.

In [13]:

fig = sns.histplot(data = tx_table, x = tx_table['n_topic'], stat = "count", discrete = True)

Let's now group the txs based on the contract name and see on a pie chart wich are the 20 most called contracts. 

In [14]:
tx_table[['contract_add','tx_hash','n_topic']].groupby(by = 'contract_add').count().sort_values(by ='tx_hash', ascending = False)

In [15]:


#define data
data = tx_table['contract_add'].value_counts().values[:20]
labels = tx_table['contract_add'].value_counts().index[:20]
red_l = [el[:5] for el in labels]

#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:20]

#create pie chart
plt.pie(data, labels = red_l, colors = colors, autopct='%.0f%%')
plt.show()

## Statistics on Tx

Most of the transactions are simple ETH Transfers. The first most called contract is Tether ERC20 contract. The second most called is https://opensea.io/collection/moondust-official

We might study some of the transactions interacting with this contract.

In [16]:
moon_dust = tx_table[tx_table['contract_add'] == '0x0B3B95547A22bee3c03Be558eC649dbd69Af8476']

In [17]:
moon_dust

#### Most topic_0 are '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'

This is a transfer of a MoonDust Token from one address to another https://etherscan.io/tx/0xab1267fc1da5ac779b4472dd4391e53b25495f8c24a5b9499b5cef84b53f62f3#eventlog

The second event is an setApprovalForAll event.
https://etherscan.io/tx/0x9cbfac0ba898699283762819e683d7f080f1f1754784a846dd094c804ddf04c7

In [18]:
#define data
data = tx_table['topic_0'].value_counts().values[:20]
labels = tx_table['topic_0'].value_counts().index[:20]
red_l = [el[:5] for el in labels]

#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:20]

#create pie chart
plt.pie(data, labels = red_l, colors = colors, autopct='%.0f%%')
plt.show()

### Analysis of a specific transaction

In this section we will dive deeper in one single transaction and parse the logs.

In [19]:
receipt = w3.eth.get_transaction_receipt('0xa803451a0501e0ec4785fdd72deb6cba1c1cbe3675169b4f7b0f0d5e547c9c51')
r_input = w3.eth.get_transaction('0xa803451a0501e0ec4785fdd72deb6cba1c1cbe3675169b4f7b0f0d5e547c9c51')

In [20]:
#get the logs
if len(receipt['logs']):
    log = receipt['logs'][0]
else:
    print('no contract interaction')

In [21]:
log

In [22]:
#get the smart contract address in the transaction
smart_contract = log['address']

In [23]:
smart_contract

In order to interpret the ABI related to the specific smartcontract we need an API key from etherscan.

In [24]:
API_KEY_me = ''

In [25]:
import requests
def get_abi(contract: str, API_KEY: str):
    url = f'https://api.etherscan.io/api?module=contract&action=getabi&address={contract}&apikey={API_KEY}'
    r = requests.get(url)
    data_as_dict = r.json()

    return(data_as_dict)

In [26]:
#get the abi for the specific contract
abi_dict = get_abi(smart_contract, API_KEY_me)

In [27]:
abi_dict

In [28]:
#in result we can read all functions call and event emitted
contract = w3.eth.contract(smart_contract, abi=abi_dict["result"])

The next command gives the called event for that specific transaction. In this case a "setApprovalForAll" event.

In [29]:
stringiii = r_input['input']
contract.decode_function_input(stringiii)

In [30]:
receipt_event_signature_hex = w3.toHex(log["topics"][0])

In particular this the signature

In [31]:
receipt_event_signature_hex

In [32]:
abi_events = [abi for abi in contract.abi if abi["type"] == "event"]

In [33]:
abi_events

In [34]:
for event in abi_events:
    # Get event signature components
    name = event["name"]
    inputs = [param["type"] for param in event["inputs"]]
    inputs = ",".join(inputs)
    # Hash event signature
    event_signature_text = f"{name}({inputs})"
    event_signature_hex = w3.toHex(w3.keccak(text=event_signature_text))
    # Find match between log's event signature and ABI's event signature
    if event_signature_hex == receipt_event_signature_hex:
        # Decode matching log
        decoded_logs = contract.events[event["name"]]().processReceipt(receipt)

In [35]:
decoded_logs